In [163]:
import nltk
import sklearn
import csv
import numpy as np
import pandas as pd
import textblob
import numpy
from nltk.corpus import sentence_polarity
import os
import sys
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.metrics import ConfusionMatrix
twtokenizer = TweetTokenizer()
import re
import math
from nltk.classify.scikitlearn import SklearnClassifier 
from sklearn.naive_bayes import MultinomialNB

In [9]:
pwd

'/Users/ayesha92ahmad/Downloads/School-work/NLP/Homework/final-project'

In [31]:
fieldnames = ["id1", "id2","pred_class","tweet_text"]
dataset=pd.read_csv("SemEval2014TweetData/corpus/downloaded-tweeti-b.dev.dist.tsv", sep='\t',names = fieldnames)
print(len(dataset))
dataset.head(10)

1654


,id1,id2,pred_class,tweet_text
0,260097528899452929,595739778,neutral,"Won the match #getin . Plus, tomorrow is a ver..."
1,263791921753882624,83619901,neutral,Some areas of New England could see the first ...
2,264194578381410304,805646850,negative,Not Available
3,264041328420204544,380148734,neutral,Not Available
4,263816256640126976,473936453,objective-OR-neutral,Not Available
5,263722278712393728,917435460,positive,Not Available
6,262936443171778560,55572169,positive,Not Available
7,260486470828171265,44399968,objective-OR-neutral,Tina Fey &amp; Amy Poehler are hosting the Gol...
8,262968617233162240,757209662,positive,Lunch from my new Lil spot ...THE COTTON BOWL ...
9,263790847424880641,420538325,positive,SNC Halloween Pr. Pumped. Let's work it for Su...


In [45]:
#filter out not available
dataset= (dataset[dataset['tweet_text']!="Not Available"])

In [46]:
np_tweets =dataset.as_matrix(columns=["tweet_text"])
np_pred= dataset.as_matrix(columns=["pred_class"])
print(np_tweets.shape)
print(np_pred.shape)

(1413, 1)
(1413, 1)


In [64]:
f = open('SemEval2014TweetData/corpus/downloaded-tweeti-b.dev.dist.tsv', 'r')
stopwords = open('SemEval2014TweetData/stopwords_twitter.txt', 'r')
stoptokens = nltk.word_tokenize(stopwords.read())
  # loop over lines in the file and use the first limit of them
  #    assuming that the tweets are sufficiently randomized
tweetdata = []
for line in f:
    if (len(tweetdata) < 10000):
    # remove final end of line character
        line = line.strip()
  # each line has 4 items separated by tabs
  # ignore the tweet and user ids, and keep the sentiment and tweet text
        tweetdata.append(line.split('\t')[2:4])

for tweet in tweetdata[:10]:
    print (tweet)

# create list of tweet documents as (list of words, label)
# where the labels are condensed to just 3:  'pos', 'neg', 'neu'
tweetdocs = []
# add all the tweets except the ones whose text is Not Available
for tweet in tweetdata:
    if (tweet[1] != 'Not Available'):
      # run the tweet tokenizer on the text string - returns unicode tokens, so convert to utf8
        tokens = twtokenizer.tokenize(tweet[1])
        if tweet[0] == '"positive"':
            label = 'pos'
        else:
            if tweet[0] == '"negative"':
                label = 'neg'
            else:
                if (tweet[0] == '"neutral"') or (tweet[0] == '"objective"') or (tweet[0] == '"objective-OR-neutral"'):
                    label = 'neu'
                else:
                    label = ''
        tweetdocs.append((tokens, label))
  
  # print a few
print(tweetdocs[:4])    


# for tweet,label in tweetdocs[:10]:
#     print(tweet)

 

['"neutral"', "Won the match #getin . Plus, tomorrow is a very busy day, with Awareness Day's and debates. Gulp. Debates..."]
['"neutral"', 'Some areas of New England could see the first flakes of the season Tuesday.']
['"negative"', 'Not Available']
['"neutral"', 'Not Available']
['"objective-OR-neutral"', 'Not Available']
['"positive"', 'Not Available']
['"positive"', 'Not Available']
['"objective-OR-neutral"', 'Tina Fey &amp; Amy Poehler are hosting the Golden Globe awards on January 13. What do you think?']
['"positive"', 'Lunch from my new Lil spot ...THE COTTON BOWL ....pretty good#1st#time#will be going back# http://t.co/Dbbj8xLZ']
['"positive"', "SNC Halloween Pr. Pumped. Let's work it for Sunday....Packers vs....who knows or caresn. #SNC #cheerpracticeonhalloween"]
[(['Won', 'the', 'match', '#getin', '.', 'Plus', ',', 'tomorrow', 'is', 'a', 'very', 'busy', 'day', ',', 'with', 'Awareness', "Day's", 'and', 'debates', '.', 'Gulp', '.', 'Debates', '...'], 'neu'), (['Some', 'areas'

In [41]:
# function that takes a word and returns true if it consists only
#   of non-alphabetic characters  (assumes import re)
def alpha_filter(w):
  # pattern to match word of non-alphabetical characters
    pattern = re.compile('^[^a-z]+$')
    if (pattern.match(w)):
        return True
    else:
        return False

In [112]:
# first preprocessing and then possibly filter tokens
processed_tweets=[]
for tweet,label in tweetdocs:
    x= [w.lower() for w in tweet]
    x= [w for w in x if not alpha_filter(w)]
    x= [w for w in x if not w in stoptokens]
    processed_tweets.append((x,label))
print(processed_tweets[:20])   

[(['won', 'match', '#getin', 'plus', 'tomorrow', 'busy', 'day', 'awareness', "day's", 'debates', 'gulp', 'debates'], 'neu'), (['areas', 'new', 'england', 'see', 'first', 'flakes', 'season', 'tuesday'], 'neu'), (['tina', 'fey', 'amy', 'poehler', 'hosting', 'golden', 'globe', 'awards', 'january', 'think'], 'neu'), (['lunch', 'new', 'lil', 'spot', 'cotton', 'bowl', 'pretty', 'good', '#1st', '#time', '#will', 'going', 'back', 'http://t.co/dbbj8xlz'], 'pos'), (['snc', 'halloween', 'pr', 'pumped', 'work', 'sunday', 'packers', 'vs', 'knows', 'caresn', '#snc', '#cheerpracticeonhalloween'], 'pos'), (['@jacquelinemegan', 'sorry', 'heart', 'paris', 'no', 'longer', 'available', 'rockwell', 'branch', 'may', 'call', 'get', 'copy', 'transferred'], 'neg'), (['manchester', 'united', 'will', 'try', 'return', 'winning', 'ways', 'face', 'arsenal', 'premier', 'league', 'old', 'trafford', 'saturday'], 'neu'), (['going', 'bulls', 'game', 'aaliyah', 'hope', 'next', 'thursday'], 'neu'), (['toon', 'fans', 'spar

In [113]:
# continue as usual to get all words and create word features

# get all words from all tweet and put into a frequency distribution
#   note lowercase, but no stemming or stopwords
all_words_list = [word for (tweet,label) in tweetdocs for word in tweet]

print(all_words_list[:100])
all_words = nltk.FreqDist(all_words_list)
# get the 2000 most frequently appearing keywords in the corpus
word_items = all_words.most_common(2000)
word_features = [word for (word,count) in word_items]
  

['Won', 'the', 'match', '#getin', '.', 'Plus', ',', 'tomorrow', 'is', 'a', 'very', 'busy', 'day', ',', 'with', 'Awareness', "Day's", 'and', 'debates', '.', 'Gulp', '.', 'Debates', '...', 'Some', 'areas', 'of', 'New', 'England', 'could', 'see', 'the', 'first', 'flakes', 'of', 'the', 'season', 'Tuesday', '.', 'Tina', 'Fey', '&', 'Amy', 'Poehler', 'are', 'hosting', 'the', 'Golden', 'Globe', 'awards', 'on', 'January', '13', '.', 'What', 'do', 'you', 'think', '?', 'Lunch', 'from', 'my', 'new', 'Lil', 'spot', '...', 'THE', 'COTTON', 'BOWL', '...', 'pretty', 'good', '#1st', '#time', '#will', 'be', 'going', 'back', '#', 'http://t.co/Dbbj8xLZ', 'SNC', 'Halloween', 'Pr', '.', 'Pumped', '.', "Let's", 'work', 'it', 'for', 'Sunday', '...', 'Packers', 'vs', '...', 'who', 'knows', 'or', 'caresn', '.']


In [114]:
# feature sets from a feature definition function

# define features (keywords) of a document for a BOW/unigram baseline
# each feature is 'contains(keyword)' and is true or false depending
# on whether that keyword is in the document
def document_features(document, word_features):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features


In [115]:
# get features sets for a document, including keyword features and category feature
featuresets = [(document_features(d, word_features), c) for (d, c) in processed_tweets]

In [116]:
# train and test a classifier
num_docs = len(featuresets)
#to create 80-10 split
train_set, test_set = featuresets[:math.floor(0.8*num_docs)],featuresets[math.floor(0.8*num_docs):]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [117]:
print("Train set Accuracy: "+ str(nltk.classify.accuracy(classifier, train_set)))
print("Test set Accuracy: "+ str(nltk.classify.accuracy(classifier, test_set)))
# show most informative features
print(classifier.show_most_informative_features(20))

Train set Accuracy: 0.8619469026548673
Test set Accuracy: 0.6042402826855123
Most Informative Features
          contains(best) = True              pos : neu    =     12.6 : 1.0
         contains(can't) = True              neg : neu    =     12.3 : 1.0
          contains(wait) = True              pos : neg    =      9.5 : 1.0
          contains(gets) = True              neg : neu    =      9.2 : 1.0
          contains(love) = True              pos : neu    =      8.5 : 1.0
        contains(pretty) = True              neg : neu    =      7.2 : 1.0
          contains(hate) = True              neg : pos    =      6.9 : 1.0
           contains(bad) = True              neg : pos    =      6.9 : 1.0
         contains(c'mon) = True              neg : pos    =      6.9 : 1.0
          contains(yeah) = True              pos : neu    =      6.7 : 1.0
         contains(power) = True              neg : neu    =      6.4 : 1.0
        contains(didn't) = True              neg : neu    =      6.4 : 1

In [118]:
# get features sets for a document, including keyword features and category feature
featuresets = [(document_features(d, word_features), c) for (d, c) in tweetdocs]

In [119]:
# train and test a classifier
num_docs = len(featuresets)
#to create 90-10 split
train_set, test_set = featuresets[:math.floor(0.9*num_docs)],featuresets[math.floor(0.9*num_docs):]
classifier = nltk.NaiveBayesClassifier.train(train_set)


In [120]:
    print("Train set Accuracy: "+ str(nltk.classify.accuracy(classifier, train_set)))
    print("Test set Accuracy: "+ str(nltk.classify.accuracy(classifier, test_set)))
    # show most informative features
    print(classifier.show_most_informative_features(20))

Train set Accuracy: 0.8977183320220299
Test set Accuracy: 0.6408450704225352
Most Informative Features
            contains(:() = True              neg : neu    =     20.9 : 1.0
       contains(excited) = True              pos : neu    =     12.0 : 1.0
         contains(can't) = True              neg : neu    =     11.7 : 1.0
          contains(best) = True              pos : neu    =     10.9 : 1.0
          contains(wait) = True              pos : neg    =     10.0 : 1.0
          contains(gets) = True              neg : neu    =      9.4 : 1.0
         contains(don't) = True              neg : neu    =      9.4 : 1.0
          contains(Good) = True              pos : neu    =      9.3 : 1.0
         contains(Honey) = True              neg : pos    =      8.1 : 1.0
         contains(C'mon) = True              neg : neu    =      7.9 : 1.0
          contains(SOPA) = True              neg : neu    =      7.9 : 1.0
          contains(love) = True              pos : neu    =      7.5 : 1

In [121]:
####   adding features from a sentiment lexicon   ####
# First look at the program in the file Subjectivity.py to load the subjectivity lexicon
# copy and paste the definition of the readSubjectivity function

# this function returns a dictionary where you can look up words and get back 
#     the four items of subjectivity information described above
def readSubjectivity(path):
    flexicon = open(path, 'r')
    # initialize an empty dictionary
    sldict = { }
    for line in flexicon:
        fields = line.split()   # default is to split on whitespace
        # split each field on the '=' and keep the second part as the value
        strength = fields[0].split("=")[1]
        word = fields[2].split("=")[1]
        posTag = fields[3].split("=")[1]
        stemmed = fields[4].split("=")[1]
        polarity = fields[5].split("=")[1]
        if (stemmed == 'y'):
            isStemmed = True
        else:
            isStemmed = False
        # put a dictionary entry with the word as the keyword
        #     and a list of the other values
        sldict[word] = [strength, posTag, isStemmed, polarity]
    return sldict

In [122]:
# create a path to where the subjectivity file resides on your disk

# create your own path to the subjclues file
SLpath = "/Users/ayesha92ahmad/Downloads/School-work/NLP/Codes/LabExercises/subjclueslen1-HLTEMNLP05.tff"
SL = readSubjectivity(SLpath)

# how many words are in the dictionary
len(SL.keys())

6885

In [123]:
# define features that include word counts of subjectivity words
# negative feature will have number of weakly negative words +
#    2 * number of strongly negative words
# positive feature has similar definition
#    not counting neutral words
def SL_features(document, word_features, SL):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    # count variables for the 4 classes of subjectivity
    weakPos = 0
    strongPos = 0
    weakNeg = 0
    strongNeg = 0
    for word in document_words:
        if word in SL:
            strength, posTag, isStemmed, polarity = SL[word]
            if strength == 'weaksubj' and polarity == 'positive':
                weakPos += 1
            if strength == 'strongsubj' and polarity == 'positive':
                strongPos += 1
            if strength == 'weaksubj' and polarity == 'negative':
                weakNeg += 1
            if strength == 'strongsubj' and polarity == 'negative':
                strongNeg += 1
            features['positivecount'] = weakPos + (2 * strongPos)
            features['negativecount'] = weakNeg + (2 * strongNeg)      
    return features

In [124]:
SL_featuresets = [(SL_features(d, word_features, SL), c) for (d, c) in tweetdocs]
num_docs = len(featuresets)
#to create 90-10 split
train_set, test_set = SL_featuresets[:math.floor(0.9*num_docs)],SL_featuresets[math.floor(0.9*num_docs):]
# retrain the classifier using these features
classifier = nltk.NaiveBayesClassifier.train(train_set)


In [125]:
print("Train set Accuracy: "+ str(nltk.classify.accuracy(classifier, train_set)))
print("Test set Accuracy: "+ str(nltk.classify.accuracy(classifier, test_set)))
# show most informative features
print(classifier.show_most_informative_features(20))

Train set Accuracy: 0.8977183320220299
Test set Accuracy: 0.6549295774647887
Most Informative Features
            contains(:() = True              neg : neu    =     20.9 : 1.0
           negativecount = 3                 neg : pos    =     12.1 : 1.0
       contains(excited) = True              pos : neu    =     12.0 : 1.0
         contains(can't) = True              neg : neu    =     11.7 : 1.0
          contains(best) = True              pos : neu    =     10.9 : 1.0
          contains(wait) = True              pos : neg    =     10.0 : 1.0
          contains(gets) = True              neg : neu    =      9.4 : 1.0
         contains(don't) = True              neg : neu    =      9.4 : 1.0
          contains(Good) = True              pos : neu    =      9.3 : 1.0
         contains(Honey) = True              neg : pos    =      8.1 : 1.0
         contains(C'mon) = True              neg : neu    =      7.9 : 1.0
          contains(SOPA) = True              neg : neu    =      7.9 : 1

In [126]:
# this function takes a document list of words and returns a feature dictionary
# it runs the default pos tagger (the Stanford tagger) on the document
#   and counts 4 types of pos tags to use as features
def POS_features(document, word_features):
    document_words = set(document)
    tagged_words = nltk.pos_tag(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    numNoun = 0
    numVerb = 0
    numAdj = 0
    numAdverb = 0
    for (word, tag) in tagged_words:
        if tag.startswith('N'): numNoun += 1
        if tag.startswith('V'): numVerb += 1
        if tag.startswith('J'): numAdj += 1
        if tag.startswith('R'): numAdverb += 1
    features['nouns'] = numNoun
    features['verbs'] = numVerb
    features['adjectives'] = numAdj
    features['adverbs'] = numAdverb
    return features

In [127]:
POS_featuresets = [(POS_features(d, word_features), c) for (d, c) in tweetdocs]
# train and test the classifier
num_docs = len(featuresets)
#to create 90-10 split
train_set, test_set = POS_featuresets[:math.floor(0.9*num_docs)],POS_featuresets[math.floor(0.9*num_docs):]
classifier = nltk.NaiveBayesClassifier.train(train_set)


In [128]:
print("Train set Accuracy: "+ str(nltk.classify.accuracy(classifier, train_set)))
print("Test set Accuracy: "+ str(nltk.classify.accuracy(classifier, test_set)))
# show most informative features
print(classifier.show_most_informative_features(20))

Train set Accuracy: 0.8859166011014948
Test set Accuracy: 0.6267605633802817
Most Informative Features
            contains(:() = True              neg : neu    =     20.9 : 1.0
       contains(excited) = True              pos : neu    =     12.0 : 1.0
         contains(can't) = True              neg : neu    =     11.7 : 1.0
          contains(best) = True              pos : neu    =     10.9 : 1.0
          contains(wait) = True              pos : neg    =     10.0 : 1.0
          contains(gets) = True              neg : neu    =      9.4 : 1.0
         contains(don't) = True              neg : neu    =      9.4 : 1.0
          contains(Good) = True              pos : neu    =      9.3 : 1.0
         contains(Honey) = True              neg : pos    =      8.1 : 1.0
         contains(C'mon) = True              neg : neu    =      7.9 : 1.0
          contains(SOPA) = True              neg : neu    =      7.9 : 1.0
          contains(love) = True              pos : neu    =      7.5 : 1

In [150]:
def CV_Scores(classifier_type, test_set):
    from collections import Counter
    from nltk.metrics import ConfusionMatrix
    refList = []
    testList = []
    for features, label in test_set:
        refList.append(label)
        testList.append(classifier_type.classify(features))
    cm = ConfusionMatrix(refList, testList)
    true_positives = Counter()
    false_negatives = Counter()
    false_positives = Counter()
    for i in refList:
        for j in refList:
            if i == j:
                true_positives[i] += cm[i,j]
            else:
                false_negatives[i] += cm[i,j]
                false_positives[j] += cm[i,j]
    #print "Confusion Matrix:"
    #print cm
    precision=0
    recall=0
    fscore=0
    for i in sorted(refList):
        if true_positives[i] == 0:
            fscore = 0
        else:
            precision = true_positives[i] / float(true_positives[i]+false_positives[i])
            recall = true_positives[i] / float(true_positives[i]+false_negatives[i])
            fscore = 2 * (precision * recall) / float(precision + recall)
    return precision, recall , fscore
    

In [151]:
## cross-validation ##
# this function takes the number of folds, the feature sets
# it iterates over the folds, using different sections for training and testing in turn
#   it prints the accuracy for each fold and the average accuracy at the end
def cross_validation_accuracy(num_folds, featuresets):
    subset_size = int(len(featuresets)/num_folds)
    accuracy_train_list = []
    accuracy_test_list = []
    presion_list=[]
    recall_list=[]
    fscore_list=[]
    # iterate over the folds
    for i in range(num_folds):
        test_this_round = featuresets[(i*subset_size):][:subset_size]
        train_this_round = featuresets[:(i*subset_size)] + featuresets[((i+1)*subset_size):]
        # train using train_this_round
        classifier = nltk.NaiveBayesClassifier.train(train_this_round)
        # evaluate against test_this_round and save accuracy
        accuracy_train_this_round = nltk.classify.accuracy(classifier, train_this_round)
        accuracy_test_this_round = nltk.classify.accuracy(classifier, test_this_round)
        accuracy_train_list.append(accuracy_train_this_round)
        accuracy_test_list.append(accuracy_test_this_round)
        precision, recall,fscore =CV_Scores(classifier, test_set)
        presion_list.append(precision)
        recall_list.append(recall)
        fscore_list.append(fscore)
    # find mean accuracy over all rounds
    print ('mean train accuracy', sum(accuracy_train_list) / num_folds)
    print ('mean test accuracy', sum(accuracy_test_list) / num_folds)
    print ('mean precision', sum(presion_list) / num_folds)
    print ('mean recall', sum(recall_list) / num_folds)
    print ('mean fscore', sum(fscore_list) / num_folds)
    

In [152]:
#kfold on SL feature
cross_validation_accuracy(10,SL_featuresets)


mean train accuracy 0.8945754716981134
mean test accuracy 0.5943262411347517
mean precision 0.9241287740272014
mean recall 0.9239049496531809
mean fscore 0.9228843189217599


In [153]:
#Kfold on POS feature
cross_validation_accuracy(10,POS_featuresets )

mean train accuracy 0.8907232704402516
mean test accuracy 0.5836879432624114
mean precision 0.9523414224100055
mean recall 0.8972381841093219
mean fscore 0.9234335668853223


In [154]:
#Kfold on POS feature
cross_validation_accuracy(10,featuresets )

mean train accuracy 0.896305031446541
mean test accuracy 0.5780141843971631
mean precision 0.9241287740272014
mean recall 0.9239049496531809
mean fscore 0.9228843189217599


In [166]:
## cross-validation ##
# this function takes the number of folds, the feature sets
# it iterates over the folds, using different sections for training and testing in turn
#   it prints the accuracy for each fold and the average accuracy at the end
def cross_validation_accuracy_SKlearn(num_folds, featuresets):
    subset_size = int(len(featuresets)/num_folds)
    accuracy_train_list = []
    accuracy_test_list = []
    presion_list=[]
    recall_list=[]
    fscore_list=[]
    # iterate over the folds
    for i in range(num_folds):
        test_this_round = featuresets[(i*subset_size):][:subset_size]
        train_this_round = featuresets[:(i*subset_size)] + featuresets[((i+1)*subset_size):]
        # train using train_this_round
        classifier = SklearnClassifier(MultinomialNB())
        classifier.train(train_this_round)
        
        # evaluate against test_this_round and save accuracy
        accuracy_train_this_round = nltk.classify.accuracy(classifier, train_this_round)
        accuracy_test_this_round = nltk.classify.accuracy(classifier, test_this_round)
        accuracy_train_list.append(accuracy_train_this_round)
        accuracy_test_list.append(accuracy_test_this_round)
        precision, recall,fscore =CV_Scores(classifier, test_set)
        presion_list.append(precision)
        recall_list.append(recall)
        fscore_list.append(fscore)
    # find mean accuracy over all rounds
    print ('mean train accuracy', sum(accuracy_train_list) / num_folds)
    print ('mean test accuracy', sum(accuracy_test_list) / num_folds)
    print ('mean precision', sum(presion_list) / num_folds)
    print ('mean recall', sum(recall_list) / num_folds)
    print ('mean fscore', sum(fscore_list) / num_folds)

In [167]:
#Kfold on document feature on multinomial naive bayes classifier
cross_validation_accuracy_SKlearn(10,featuresets )


mean train accuracy 0.8875786163522014
mean test accuracy 0.5794326241134751
mean precision 0.9245023014193092
mean recall 0.9382635383635284
mean fscore 0.9307603702136215
